# *Conexión al Drive*

In [1]:
# prompt: crear conexion a drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import sys
path='/content/drive/MyDrive/cod/marketing/MarketingprojectG8/data'
os.chdir(path)###volver la carpeta del repositorio directorio de trabajo
sys.path.append(path)### agregarla al path, para poder leer los archivos de funciones propios como paquetes

# *Librerias*

In [3]:
###paquete para crear y trabajar con bases de datos.
import sqlite3 as sql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
#import a_funciones as fn
from mlxtend.preprocessing import TransactionEncoder
import re

# *Conexión a SQL*

In [4]:
# Conectarse a la base de datos
con=sql.connect('db_movies')

In [5]:
# Crear un cursor
cur=con.cursor()### otro tipo de conexión para ejecutar consultas en la bd sin traer ni llevar información

In [6]:
# Obtener una lista de tablas en la base de datos
cur.execute("""SELECT name FROM sqlite_master WHERE type= 'table'; """)
tables = cur.fetchall()
print("Tablas disponibles:", tables)

Tablas disponibles: [('ratings',), ('movies',), ('movies_new',)]


### *Cargar las tablas en DataFrames*

In [7]:
# Cargar tabla movies
cur.execute('SELECT * FROM movies')
movies_data = cur.fetchall()
movies_columns = [description[0] for description in cur.description]  # Obtener nombres de columnas
movies_df = pd.DataFrame(movies_data, columns=movies_columns)

# Mostrar las primeras filas de la tabla
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
# Cargar tabla ratings
cur.execute('SELECT * FROM ratings')
ratings_data = cur.fetchall()
ratings_columns = [description[0] for description in cur.description]  # Obtener nombres de columnas
ratings_df = pd.DataFrame(ratings_data, columns=ratings_columns)

# Mostrar las primeras filas de la tabla
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## *Nueva Tabla Movies*





In [9]:
# Crear la nueva tabla con las columnas 'movie_name', 'year' y 'genres'
cur.execute("""
CREATE TABLE IF NOT EXISTS movies_new (
    movieId INTEGER PRIMARY KEY,
    movie_name TEXT,
    year INTEGER,
    genres TEXT
);
""")

# Insertar los datos de la tabla original 'movies' a la nueva tabla 'movies_new'
cur.execute("""
INSERT INTO movies_new (movieId, movie_name, year, genres)
SELECT
    movieId,
    TRIM(SUBSTR(title, 1, LENGTH(title) - 6)) AS movie_name,  -- Extraemos el nombre de la película
    CAST(SUBSTR(title, LENGTH(title) - 4, 4) AS INTEGER) AS year,  -- Extraemos el año
    genres
FROM
    movies;
""")

# Consultar las primeras filas de la nueva tabla para verificar los datos
cur.execute("SELECT movieId, movie_name, year, genres FROM movies_new LIMIT 10;")
data = cur.fetchall()

# Mostrar los resultados
for row in data:
    print(row)

(1, 'Toy Story', 1995, 'Adventure|Animation|Children|Comedy|Fantasy')
(2, 'Jumanji', 1995, 'Adventure|Children|Fantasy')
(3, 'Grumpier Old Men', 1995, 'Comedy|Romance')
(4, 'Waiting to Exhale', 1995, 'Comedy|Drama|Romance')
(5, 'Father of the Bride Part II', 1995, 'Comedy')
(6, 'Heat', 1995, 'Action|Crime|Thriller')
(7, 'Sabrina', 1995, 'Comedy|Romance')
(8, 'Tom and Huck', 1995, 'Adventure|Children')
(9, 'Sudden Death', 1995, 'Action')
(10, 'GoldenEye', 1995, 'Action|Adventure|Thriller')


In [10]:
# Ejecutar la consulta SQL
data = "SELECT movieId, movie_name, year, genres FROM movies_new LIMIT 10;"

# Cargar los resultados en un DataFrame
movie_df = pd.read_sql(data, con)

# Mostrar los primeros resultados
movie_df.head()

,movieId,movie_name,year,genres
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,1995,Adventure|Children|Fantasy
2,3,Grumpier Old Men,1995,Comedy|Romance
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance
4,5,Father of the Bride Part II,1995,Comedy


## *Nueva Tabla Ratings*

In [11]:
# Crear una nueva columna 'ratings_times' en la tabla 'ratings'
cur.execute("ALTER TABLE ratings ADD COLUMN ratings_times TEXT;")

# Actualizar la columna 'ratings_times' con el valor convertido de 'timestamp'
cur.execute("""
UPDATE ratings
SET ratings_times = datetime(timestamp, 'unixepoch');
""")

# Consultar las primeras filas para verificar que 'ratings_times' esté correcto
cur.execute("SELECT movieId, userId, rating, ratings_times FROM ratings LIMIT 10;")
data = cur.fetchall()

# Mostrar los resultados
for row in data:
    print(row)

(1, 1, 4.0, '2000-07-30 18:45:03')
(3, 1, 4.0, '2000-07-30 18:20:47')
(6, 1, 4.0, '2000-07-30 18:37:04')
(47, 1, 5.0, '2000-07-30 19:03:35')
(50, 1, 5.0, '2000-07-30 18:48:51')
(70, 1, 3.0, '2000-07-30 18:40:00')
(101, 1, 5.0, '2000-07-30 18:14:28')
(110, 1, 4.0, '2000-07-30 18:36:16')
(151, 1, 5.0, '2000-07-30 19:07:21')
(157, 1, 5.0, '2000-07-30 19:08:20')


In [12]:
# Consulta SQL para obtener los datos de la tabla 'ratings' (incluyendo 'ratings_times')
data = """
SELECT movieId, userId, rating, ratings_times
FROM ratings
LIMIT 10;
"""

# Cargar los resultados de la consulta directamente en un DataFrame
ratings_df = pd.read_sql(data, con)

# Mostrar los primeros resultados
ratings_df.head()

,movieId,userId,rating,ratings_times
0,1,1,4.0,2000-07-30 18:45:03
1,3,1,4.0,2000-07-30 18:20:47
2,6,1,4.0,2000-07-30 18:37:04
3,47,1,5.0,2000-07-30 19:03:35
4,50,1,5.0,2000-07-30 18:48:51


# *Cerra la conexion*

In [ ]:
# Cerrar la conexión
#cur.close()
#con.close()

# *Análisis exploratorio de los datos*

### *Verificar valores nulos en ambas tablas*

In [13]:
# Verificar valores nulos en las tablas
print("Valores nulos en movies:")
movies_df.isnull().sum()

Valores nulos en movies:


,0
movieId,0
title,0
genres,0


In [14]:
print("Valores nulos en ratings:")
ratings_df.isnull().sum()

Valores nulos en ratings:


,0
movieId,0
userId,0
rating,0
ratings_times,0


### *Análisis de distribuciones y actividad de usuarios*

#### *Distribución de calificaciones*

In [ ]:
# Histograma de las calificaciones
ratings_df['rating'].hist(bins=10)
plt.title("Distribución de Calificaciones")
plt.xlabel("Calificación")
plt.ylabel("Frecuencia")
plt.show()


### *¿Cuántas calificaciones han hecho los usuarios?*

In [ ]:
# Ver usuarios más activos
top_users = ratings_df['userId'].value_counts().head(10)
top_users.plot(kind='bar')
plt.title("Usuarios Más Activos")
plt.xlabel("Usuario")
plt.ylabel("Número de Calificaciones")
plt.show()


In [ ]:
cr=pd.read_sql(""" select
                          "Book-Rating" as rating,
                          count(*) as conteo
                          from book_ratings
                          group by "Book-Rating"
                          order by conteo desc""", conn)